In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
!pip install sweetviz
import sweetviz as sv

Mounted at /content/drive
     |████████████████████████████████| 15.1MB 481kB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()
train_err2 = train_err.copy()
test_err2 = test_err.copy()

# 퀼리티 3,4는 값이 1밖에 없어서 의미가 없기 때문에 제외
# 펌웨어 버전은 이미 err 자료에 있기 때문에 중복으로 제외
# 퀼리티 0은 EDA 결과 퀼리티 2와 피어슨 상관계수 0.99를 보이기 때문에 0과 2 중에 하나만 골랐음
# 0의 결측치 Nan은 15%고 2의 결측치는 4%기 때문에 결측치가 적은 2를 남기고 0은 drop 시킨다.

## 퀼리티 6과 7도 0.81로 강한 상관관계를 보이고 있는데, 하나를 제외해야 하나 고민이 됨
train_qui2.drop(['fwver','quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)
test_qui2.drop(['fwver', 'quality_0', 'quality_3', 'quality_4'], axis =1, inplace=True)

test_qui2 = test_qui2.replace(',','', regex=True)
train_qui2 = train_qui2.replace(',','', regex=True)
test_qui2.loc[:,'quality_1':'quality_12'] = test_qui2.loc[:,'quality_1':'quality_12'].astype(float)
train_qui2.loc[:,'quality_1':'quality_12'] = train_qui2.loc[:,'quality_1':'quality_12'].astype(float)

train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')

In [6]:
# 퀼리티 임계치
# [-1, 0, 1] == -1 카테고리 -1, -1~0 =카테고리 0, 그 이상 = 카테고리 1
qualities = {}

qualities['quality_1'] = [-1,0, 1]
qualities['quality_2'] = [-1,0, 1]
qualities['quality_5'] = [-1,0,1,2,4, 10]
qualities['quality_6'] = [-1,0, 1]
qualities['quality_7'] = [0,1,100, 101]
qualities['quality_8'] = [0,1,2]
qualities['quality_9'] = [0,1,2]
qualities['quality_10'] = [0,1,2,3, 4, 5,7, 10 ,14,15] ##??
qualities['quality_11'] = [-1,0,1]
qualities['quality_12'] = [0,1,2]

## 모델 임계치
## 모델 0,1,2,3,4의 비율을 더하면 99.999% 이므로, 모델 5~8은 other model로 통합하는 것이 바람직 해 보임.

In [7]:
import warnings
warnings.filterwarnings(action='ignore')

In [8]:
def quality_categorize(dataset):
  data = dataset.copy()
  for key in tqdm(qualities):
    intervals = qualities.get(key)
    for i in range(len(intervals)):
      if intervals[i] == intervals[-1]:
        data[key][data[key] >= intervals[i]] =  intervals[i]
      # elif intervals[i+1] - intervals[i] == 1:
      #   continue
      else:
        data[key][(data[key] >= intervals[i]) & (data[key] < intervals[i+1])] = intervals[i]
  return data


In [9]:
train_qui3 = quality_categorize(train_qui2)
test_qui3 = quality_categorize(test_qui2)

In [10]:
train_err3 = train_err2[['user_id', 'model_nm', 'errtype', 'errcode', 'time']]
test_err3 = test_err2[['user_id', 'model_nm', 'errtype', 'errcode', 'time']]

In [11]:
# 에러 코드의 분포는 다음과 같다.

print(train_err2.errcode.value_counts())
# 중간에 나오다가 잘리지만.. 전에 발간한 리포트를 보면 상위 15개 에러가 99.9%를 차지하고 있다.
# 따라서 상위 15개는 각각의 카테고리로 주고, 나머지는 16번째 '기타' 카테고리로 묶으면 어떨까 싶습니다.
# 상위 15개 리스트
err_code =['1','0','connection timeout','B-A8002','80','79','14','active','2','84','85','standby','NFANDROID2','connection fail to establish','3']

# 위의 15개에 포함되지 않는 에러코드는 걍 'etc'로 분류
train_err3['errcode'][~train_err3['errcode'].isin(err_code)] = 'ETC'
test_err3['errcode'][~train_err3['errcode'].isin(err_code)] = 'ETC'


## 에러타입은 어차피 41개 뿐이라... 분포도 고른 편 같고... 얌전히 두기로 함.

1                     8906967
0                     2599123
connection timeout    1835600
B-A8002                897863
80                     334018
                       ...   
15920                       1
21245                       1
10566                       1
21885                       1
3521                        1
Name: errcode, Length: 2805, dtype: int64


In [12]:
# 1일까지 발생한 오류 ~ 2일까지 발생한 오류 ~ 3일까지 ~~~ n일까지 발생한 오류(누적)을 0,1,2,3,4,5 칼럼에 넣어주는 함수
# 퀼리티 로그의 경우 문제가 생기면 문제 이후에 2시간 분량을 수집한다고 했으므로, 몇 번이나 수집되었는지 알면 문제가 몇 번 있었는지도 알 수 있다.
# time에 관한 EDA 결과, 에러코드 수집 시작시간 10월 31일 ~ 11월 30일까지

begins = date(2020, 10, 31).timetuple().tm_yday
ends = date(2020, 11, 30).timetuple().tm_yday

q_list = []
er_list = []

In [13]:
###################### 시간 별 차이가 있는가...? ###################

In [14]:
### 혹시나 몰라서 펌웨어 날려버리기 전에, 오류 사용자와 정상 사용자의 차이를 비교
# 정상 사용자가 많이 사용하는 모델과 오류 사용자가 많이 사용하는 모델에는 차이가 있는가.
# EDA 결과는 리포트로 구글 드라이브 메인에 저장.

temp =train_err2.drop_duplicates(subset = ['user_id'])
temp = temp[['user_id', 'model_nm', 'fwver']]

prob_list = list(set(train_prob.user_id))
temp['problems'] = temp.user_id.isin(prob_list)

temp_tr = temp[temp['problems']]
temp_fls = temp[temp['problems'] != True]

temp_tr.set_index('user_id', inplace = True)
temp_fls.set_index('user_id', inplace = True)

In [15]:
train_err4 = train_err3.copy()

In [17]:
train_err4['hour'] = train_err4['time'].dt.hour
train_err4['weekday'] = train_err4['time'].dt.weekday

In [45]:
train_err4['problems'] = train_err4.user_id.isin(prob_list)
train_err4_tr = train_err4[train_err4['problems']]
train_err4_fls = train_err4[train_err4['problems'] != True]

In [51]:
train_err4_tr

,user_id,model_nm,errtype,errcode,time,hour,weekday,problems
317,10001,model_2,15,1,2020-11-01 02:04:15,2,6,True
318,10001,model_2,15,1,2020-11-01 02:04:15,2,6,True
319,10001,model_2,31,1,2020-11-01 02:04:15,2,6,True
320,10001,model_2,33,2,2020-11-01 02:04:15,2,6,True
321,10001,model_2,12,1,2020-11-01 02:04:21,2,6,True
...,...,...,...,...,...,...,...,...
16554088,24998,model_0,15,1,2020-11-29 05:08:46,5,6,True
16554089,24998,model_0,16,1,2020-11-30 13:26:04,13,0,True
16554090,24998,model_0,31,0,2020-11-30 13:26:24,13,0,True
16554091,24998,model_0,40,1,2020-11-30 21:03:51,21,0,True


In [52]:
train_err4_fls

,user_id,model_nm,errtype,errcode,time,hour,weekday,problems
0,10000,model_3,15,1,2020-11-01 02:56:16,2,6,False
1,10000,model_3,12,1,2020-11-01 03:03:09,3,6,False
2,10000,model_3,11,1,2020-11-01 03:03:09,3,6,False
3,10000,model_3,16,1,2020-11-01 05:05:14,5,6,False
4,10000,model_3,4,0,2020-11-01 05:05:15,5,6,False
...,...,...,...,...,...,...,...,...
16554658,24999,model_3,15,1,2020-11-30 16:30:51,16,0,False
16554659,24999,model_3,16,1,2020-11-30 17:26:25,17,0,False
16554660,24999,model_3,4,0,2020-11-30 17:26:25,17,0,False
16554661,24999,model_3,4,0,2020-11-30 17:26:31,17,0,False


In [48]:
train_err4_tr.drop(['time'], axis = 1)

,user_id,model_nm,errtype,errcode,hour,weekday,problems
317,10001,model_2,15,1,2,6,True
318,10001,model_2,15,1,2,6,True
319,10001,model_2,31,1,2,6,True
320,10001,model_2,33,2,2,6,True
321,10001,model_2,12,1,2,6,True
...,...,...,...,...,...,...,...
16554088,24998,model_0,15,1,5,6,True
16554089,24998,model_0,16,1,13,0,True
16554090,24998,model_0,31,0,13,0,True
16554091,24998,model_0,40,1,21,0,True


In [49]:
## 1800만개 데이터를 유저 id로 묶지 않고 불만 있음/없음으로 나눠서 그냥 각각 몇번씩 나오는지 세어 보았다.
# EX. 불만이 접수된 경우에 hour(시간)은 몇 시가 몇 번이나 있는지 등.

df = sv.compare([train_err4_tr.drop(['time'], axis = 1), "오류발생"], [train_err4_fls.drop(['time'], axis = 1), "정상"])
df.show_html("drive/My Drive/오류이용자_정상이용자_차이비교(시간대)_not_grouped2.html")


Report drive/My Drive/오류이용자_정상이용자_차이비교(시간대)_not_grouped2.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [54]:
train_err5 = train_err4[['user_id', 'hour', 'weekday']]

In [53]:
# 24시간을 단순하게 평균내면, 23시와 1시의 평균이 이상해지기 때문에(실제로는 2시간 차이인데, 산술적으로 22시간 차이로 잡힘)
# 태양광 때 처럼 일단 코사인 시간으로 바꿔서 평균, 분산, 최빈값을 구한다.

def cos_time(dataset):
  dataset['cos_time'] = np.cos(2*np.pi*dataset.hour/24)
  return dataset

In [55]:
train_err5 = cos_time(train_err5)

In [58]:
train_err5

,user_id,hour,weekday,cos_time,problems
0,10000,2,6,0.866025,False
1,10000,3,6,0.707107,False
2,10000,3,6,0.707107,False
3,10000,5,6,0.258819,False
4,10000,5,6,0.258819,False
...,...,...,...,...,...
16554658,24999,16,0,-0.500000,False
16554659,24999,17,0,-0.258819,False
16554660,24999,17,0,-0.258819,False
16554661,24999,17,0,-0.258819,False


In [57]:
train_err5['problems'] = train_err5.user_id.isin(prob_list)
train_err5_tr = train_err5[train_err5['problems']]
train_err5_fls = train_err5[train_err5['problems'] != True]

In [70]:
def cos_stats(data):
  ids = list(set(data['user_id']))
  ids.sort()
  stats = []
  for id in tqdm(ids):
    temp = data[data.user_id == id]
    if len(temp) > 1:
      a = temp.cos_time.mean()
      b = temp.cos_time.std()
      c = temp.cos_time.mode().values[0]
      t_list = [a,b,c]
    else:
      t_list = [0,0,0]
    stats.append(t_list)
  df = pd.DataFrame(stats, columns=['mean', 'std', 'freq'], index=ids)
  return df

In [71]:
### 불만있는 사람과 없는 사람,, 오류발생 시간의 평균/분산/최빈값의 차이는 존재하는가
test = cos_stats(train_err5_tr)
test2 = cos_stats(train_err5_fls)

In [76]:
# 차이를 보기 위해 리포트 생성
df = sv.compare([test, "오류발생"], [test2, "정상"])
df.show_html("drive/My Drive/오류이용자_정상이용자_시간_통계값.html")


Report drive/My Drive/오류이용자_정상이용자_시간_통계값.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [79]:
test.describe()

,mean,std,freq
count,5000.000000,5000.000000,5000.000000
mean,0.017349,0.656124,0.155173
std,0.204715,0.080714,0.567338
min,-0.836695,0.000000,-1.000000
25%,-0.117718,0.619600,-0.258819
50%,0.006817,0.668032,0.258819
75%,0.139768,0.707194,0.500000
max,0.963660,0.868919,1.000000


In [80]:
test2.describe()

,mean,std,freq
count,10000.000000,10000.000000,10000.000000
mean,0.095377,0.612915,0.264852
std,0.265754,0.144888,0.545905
min,-1.000000,0.000000,-1.000000
25%,-0.060499,0.578834,-0.258819
50%,0.083537,0.648170,0.500000
75%,0.255206,0.698329,0.707107
max,0.993070,1.003013,1.000000


In [82]:
### 같은 과정을 퀼리티 데이터로 진행해 봄 ###
train_qui4 = train_qui3.copy()
train_qui4['problems'] = train_qui4.user_id.isin(prob_list)

In [85]:
train_qui4['hour'] = train_qui4['time'].dt.hour
train_qui4['weekday'] = train_qui4['time'].dt.weekday

In [89]:
train_qui5 = cos_time(train_qui4)
train_qui5_tr = train_qui5[train_qui5['problems']]
train_qui5_fls = train_qui5[train_qui5['problems'] != True]

In [90]:
qui_tr = cos_stats(train_qui5_tr)
qui_fls = cos_stats(train_qui5_fls)

In [91]:
df = sv.compare([qui_tr, "오류발생"], [qui_fls, "정상"])
df.show_html("drive/My Drive/오류이용자_정상이용자_시간통계비교(퀼리티데이터).html")


Report drive/My Drive/오류이용자_정상이용자_시간통계비교(퀼리티데이터).html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [31]:
# 이거는 그냥 무식하게 평균으로 퉁쳐 보았을 때.. 차이가 있는지 보기 위함.

train6 = train_err5.groupby(by = 'user_id').mean()

In [29]:
train6['problems'] = train6.user_id.isin(prob_list)

In [32]:
train6_tr = train6[train6['problems']]
train6_fls = train6[train6['problems'] != True]

In [33]:
train6_tr

,hour,weekday,cos_time,problems
user_id,,,,
10001,14.962791,3.553911,-0.131223,True
10004,13.296010,2.963964,0.161332,True
10005,13.060080,3.299065,0.281449,True
10006,11.067347,2.844898,-0.128289,True
10008,3.935484,3.112903,0.508247,True
...,...,...,...,...
24983,9.055718,2.744868,0.631239,True
24984,14.846777,3.106770,0.057759,True
24990,12.555862,3.477241,0.188986,True


In [36]:
df = sv.compare([train6_tr, "오류발생"], [train6_fls, "정상"])
df.show_html("drive/My Drive/오류이용자_정상이용자_차이비교(시간대).html")


Report drive/My Drive/오류이용자_정상이용자_차이비교(시간대).html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
